## Deliverable 2. Create a Customer Travel Destinations Map.

In [55]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [56]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

In [57]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your minimum desired temperature for your vacation trip"))
max_temp = float(input("What is your maximum desired temperature for your vacation trip"))

What is your minimum desired temperature for your vacation trip70
What is your maximum desired temperature for your vacation trip75


In [58]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp)&
                                     (city_data_df["Max Temp"]>=min_temp)]

In [59]:
# 4a. Determine if there are any empty rows.
preferred_temp_df.count()

City_ID                56
City                   56
Lat                    56
Lng                    56
Max Temp               56
Humidity               56
Cloudiness             56
Wind Speed             56
Country                56
Current Description    56
dtype: int64

In [60]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_temp_df=preferred_temp_df.dropna()

In [66]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_temp_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [67]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [68]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df= hotel_df[hotel_df["Hotel Name"].str.strip().astype(bool)]
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Ocos,GT,71.60,few clouds,14.5094,-92.1933,Auto Hotel El Soñador
10,Baracoa,CU,72.99,clear sky,20.3467,-74.4958,Hostal Rio Miel
11,Mataura,NZ,74.17,light rain,-46.1927,168.8643,Ellie's Villa
12,Airai,TL,70.43,overcast clouds,-8.9266,125.4092,Juvinal Martin
31,Kapaa,US,71.60,scattered clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
57,Huarmey,PE,71.47,overcast clouds,-10.0681,-78.1522,Hotel Miramar
89,Flinders,AU,73.00,broken clouds,-34.5833,150.8552,Shellharbour Resort & Conference Centre
105,Santa Quiteria Do Maranhao,BR,73.69,clear sky,-3.5156,-42.5467,Casa Galvão
145,Nishihara,JP,71.60,scattered clouds,26.1842,127.7558,Minshuku Agaihama
166,Arraial Do Cabo,BR,73.56,light rain,-22.9661,-42.0278,Pousada Porto Praia


In [75]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [73]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [74]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))